In [31]:
# -*- coding: utf-8 -*-
"""Copy of PYTHONIC APP LOGIN

Automatically generated by Colaboratory.

Using python to log into an app and get data
"""

import pandas as pd
import requests as r
from bs4 import BeautifulSoup
# import json
import lxml
from urllib.parse import urljoin
from random import randint
from time import sleep
from datetime import datetime
from config import APP_BASE_URL,EMAIL,PASSWORD


def login_to_app():
  # Instantiate A login Session to be used for the operation
  with r.Session() as s:

    get_request = s.get(APP_BASE_URL+'/users/sign_in')

    # each get request comes with a unique auth token, hence we use bs to get this token from the page content
    login_page_content = BeautifulSoup(get_request.content,'html.parser')
    token = login_page_content.find("input", {"name":"authenticity_token"})["value"]

    # define payload within session using the token generated above
    login_payload = {
      'authenticity_token':token,
      "user[email]":EMAIL,
      "user[password]" :PASSWORD}

    login_entry = s.post(APP_BASE_URL+'/users/sign_in', data= login_payload)
    # print('login_entry.text')
    if login_entry:
      print('login to App was successful with {}'.format(login_entry))
    else:
      print('there was possibly an error')
      
  return s


def get_page_content(s):

  # Immediately after logging in, i tell the login what page i want to pull data from
  # In this case, the "reports/total_orders" page-
  # save this as a constant to call it easily
  ORDERS_PAGE_URL = APP_BASE_URL+'/reports/total_orders'

  orders_home_page= s.get(ORDERS_PAGE_URL)
  # Make a get request to get the url of the app in the default order, page 2
  orders_page_data = s.get(ORDERS_PAGE_URL+'?page=2&order_by=created_at&sort_by=ASC')

  """
  Before Using BeautifulSoup, there was need to inspect the web page to know what classes/html objects
  contained the necessary table to be scraped.
  """
  # use beautifulsoup to get the content of the orders page 
  orders_page_content =BeautifulSoup(orders_page_data.content,'lxml')
  # Then find the table tag with the class name "table table-striped"
  orders_table = orders_page_content.find("table", {"class":"table table-striped"})
  # print(orders_table)  #TO PREVIEW raw html data
  return orders_home_page,orders_page_content,orders_page_data


def extract_data_to_dataframe(s,orders_home_page,orders_page_content,orders_page_data):
  # BEGIN CREATION OF DATAFRAME
  # Write all headers to a list
  orders_home_page =orders_home_page
  orders_page_data = orders_page_data

  orders_table_headers = []
  for table_header in orders_page_content.find_all('th'):
    col_name =table_header.text
    orders_table_headers.append(col_name)
    
  # Preview the table headers to be sure all were written to the list
  # print(orders_table_headers)

  # redefining get request to dynamically add the data on more pages if they exist
  base_url_for_data = APP_BASE_URL+ f"/reports/total_orders?"
  order = '&order_by=created_at&sort_by=ASC'
  page = 0
  # Instantiate new dataframe to generate
  all_data = pd.DataFrame()
  while True:
      orders_dynamic_pages = s.get(base_url_for_data + f"page={page}" + order)
      orders_page_paginated_content = BeautifulSoup(orders_dynamic_pages.content,'lxml')

      table_data = []
      for table_row in orders_page_paginated_content.find_all('tr'):
        row = {}
        for cell_data,table_header in zip(table_row.find_all('td'),orders_table_headers):
          row[table_header] = cell_data.text.replace('\n','').strip()
        table_data.append(row)
      page_data = pd.DataFrame(table_data)
      page+=1 
      print('now extracting data from page {}'.format(page))
      if(page >=20)|len(page_data) < 1:
          break
      else:
        all_data = pd.concat([all_data,page_data])
        print('sleeping for some seconds before next request')
        sleep(randint(15,20))
  print("successfully extracted {} records from web app".format(len(all_data)))
  return all_data

def write_to_file(all_data):
  extracted_file = all_data.to_csv("extracted_records_{}.csv".format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S')))
  return extracted_file


def main():
  print('Now Running script')
  s = login_to_app()
  orders_home_page,orders_page_content,orders_page_data = get_page_content(s)
  all_data = extract_data_to_dataframe(s,orders_home_page,orders_page_content,orders_page_data)
  write_to_file(all_data)
  # return output           


if __name__ == "__main__":
	main()


# print(len(all_data))
# len(all_data)
# all_data
# # page
# all_data.head()
# # print(df)

# df

Now Running script
login to App was successful with <Response [200]>
now extracting data from page 1
sleeping for some seconds before next request
now extracting data from page 2
sleeping for some seconds before next request
now extracting data from page 3
sleeping for some seconds before next request
now extracting data from page 4
successfully extracted 150 records from web app
